# Evaluate Random Forest with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
rf = RandomForestClassifier()
param = {'n_estimators': [10,150,300],
        'max_depth': [30,60,90, None] }
gs= GridSearchCV(rf, param, cv=5, n_jobs=-1)

In [5]:
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending= False)[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,20.789289,0.364683,0.217540,0.010041,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.979372,0.977538,0.975741,0.970350,0.971249,0.974852,0.003513,1
11,36.853645,5.700083,0.258431,0.043241,None,300,"{'max_depth': None, 'n_estimators': 300}",0.978475,0.979335,0.974843,0.965858,0.973046,0.974313,0.004815,2
8,41.400910,0.272129,0.348329,0.058187,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977578,0.978437,0.974843,0.968553,0.971249,0.974133,0.003751,3
10,22.825453,0.397953,0.291913,0.072787,None,150,"{'max_depth': None, 'n_estimators': 150}",0.976682,0.976640,0.974843,0.968553,0.970350,0.973415,0.003351,4
5,27.292978,1.301447,0.268263,0.059662,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.976682,0.976640,0.973046,0.967655,0.970350,0.972876,0.003530,5


In [6]:
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending= False)[:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,20.360429,1.805283,0.222900,0.060796,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.975785,0.975741,0.974843,0.968553,0.969452,0.972876,0.003192,1
11,36.496532,1.651467,0.265888,0.044200,None,300,"{'max_depth': None, 'n_estimators': 300}",0.975785,0.976640,0.973944,0.968553,0.968553,0.972696,0.003492,2
8,33.464812,0.089223,0.276035,0.013443,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.976682,0.974843,0.974843,0.966757,0.969452,0.972517,0.003762,3
6,2.979891,0.291554,0.135923,0.032042,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.973991,0.975741,0.978437,0.964960,0.966757,0.971978,0.005223,4
4,17.198752,0.944821,0.233735,0.075120,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.975785,0.973046,0.970350,0.965858,0.970350,0.971080,0.003297,5
